In [1]:
import os, random
from pydub import AudioSegment
from xml.dom import minidom
import numpy as np
import pandas as pd
from pysndfx import AudioEffectsChain
from librosa import load
import shutil

In [2]:
f = open("D://Background_noise//meta.txt","r")
match_noise_to_type = []
content = f.readlines()

for line in content:
    y = line.split('	')
    audio_filename = y[0][6:len(y[0])]
    audio_category = y[1][0:len(y[1])-1].replace('/', '_')
    
    match_noise_to_type.append((audio_filename, audio_category))

noise_dict = dict(match_noise_to_type)
#print(noise_dict)

In [7]:
h_directory = 'D://Datasets//TRAINING//Happy//'
a_directory = 'D://Datasets//TRAINING//Angry//'
n_directory = 'D://Datasets//TRAINING//Neutral//'
s_directory = 'D://Datasets//TRAINING//Sad//'
o_directory = 'D://Datasets//TRAINING//Other//'

h_test = 'D://Datasets//TRAINING//Happy_test//'
a_test = 'D://Datasets//TRAINING//Angry_test//'
n_test = 'D://Datasets//TRAINING//Neutral_test//'
s_test = 'D://Datasets//TRAINING//Sad_test//'
o_test = 'D://Datasets//TRAINING//Other_test//'

h_EMO = 'D://Datasets//EMO-DB//wav//Happy//'
a_EMO = 'D://Datasets//EMO-DB//wav//Angry//'
n_EMO = 'D://Datasets//EMO-DB//wav//Neutral//'
s_EMO = 'D://Datasets//EMO-DB//wav//Sad//'
o_EMO = 'D://Datasets//EMO-DB//wav//Other//'

background_noise = 'D://Background_noise//TUT2016//'
background_noise_test = 'D://Background_noise//noise_test//'
background_noise_all = 'D://Background_noise//noise_all//'
background_noise_home = 'D://Background_noise//noise_home//'

emotion_train_folders = [h_directory, a_directory, n_directory, s_directory, o_directory]
emotion_test_folders = [h_test, a_test, n_test, s_test, o_test]

In [8]:
def rename_emotion_wavs(emotion_directory):
    if 'Happy' in emotion_directory:
        word = 'Happy'
    elif 'Angry' in emotion_directory:
        word = 'Angry'
    elif 'Neutral' in emotion_directory:
        word = 'Neutral'
    elif 'Sad' in emotion_directory:
        word = 'Sad'
    else:
        word = 'Other'
    
    index = 0
    for file in os.listdir(emotion_directory):
        src = emotion_directory + file
        dst = emotion_directory + word + str(index) + '.wav'
        try:
            os.rename(src, dst)
        except WindowsError:
            os.remove(dst)
            os.rename(src, dst)
        index += 1
        print(dst)
        
#rename_emotion_wavs(h_directory)
#rename_emotion_wavs(a_directory)
#rename_emotion_wavs(n_directory)
#rename_emotion_wavs(s_directory)
#rename_emotion_wavs(o_directory)

In [9]:
def rename_noise_wavs(noise_directory):
    index = 0
    for noise in os.listdir(noise_directory):
        noise_type = noise_dict.get(noise)
        new_noise_name = noise_type + '_' + str(index)
        src = noise_directory + noise
        dst = noise_directory + new_noise_name + '.wav'
        os.rename(src, dst) 
        index += 1

        print(dst)

#rename_noise_wavs(background_noise_all)

In [10]:
def delete_directory(path):
    shutil.rmtree(path, ignore_errors=True)

In [11]:
def partition_directory(path, new_path, percent):
    
    if not os.path.exists(new_path):
        os.mkdir(new_path)
    else:
        delete_directory(new_path)
        os.makdir(new_path)
        
    total = len(os.listdir(path))
    test_number = int(total * percent)
    test_set = np.random.permutation(total)[0:test_number]
    
    for index in test_set:
        try:
            if os.listdir(path)[index].endswith('.wav'):
                original_location = path + os.listdir(path)[index]
                new_location = new_path + os.listdir(path)[index]
                os.rename(original_location, new_location)
                print(new_location)
        except:
            pass

    
    print('# of training data + # of testing data = ' + str(total))
    print(str(percent) + ' of the data is in training set')
    print('# of training data = ' + str(test_set.shape))


In [13]:
def pad_silence_per_file(emotionfile, newSoundFile):
    
    emotionsound = AudioSegment.from_wav(emotionfile)
    file_duration = emotionsound.duration_seconds * 1000
    silence = AudioSegment.silent(duration=5000)
    threshold = 5000 - file_duration
    if threshold > 0:
        overlay_start = np.random.randint(0, threshold)
    else:
        emotionsound = emotionsound[0:5000]
        overlay_start = 0
    
    newSound = silence.overlay(emotionsound, position=overlay_start)
    
    newSound=newSound[0:5000]
    newSound.export(newSoundFile, format='wav')  ### save the new generated file in a folder

In [14]:
def pad_silence_per_folder(directory, extension):
    for file in os.listdir(directory):
        if file.endswith(extension) and file[0] != '.':
            newSoundFile = directory + file
            pad_silence_per_file(newSoundFile, newSoundFile)
            print(newSoundFile)

In [16]:
'''
pad_silence_per_folder(path, '.wav')

for path in emotion_test_folders:
    pad_silence_per_folder(path, '.wav')
'''
#pad_silence_per_folder(h_directory, '.wav')
#pad_silence_per_folder(h_test, '.wav')

"\npad_silence_per_folder(path, '.wav')\n\nfor path in emotion_test_folders:\n    pad_silence_per_folder(path, '.wav')\n"

In [17]:
def change_amplitude(emotionfile, d1, newSoundFile, d2):
    
    if d1 <= d2:
        sound = AudioSegment.from_file(emotionfile) - np.random.randint(0, (6 * d2/d1 - 1))
        sound.export(newSoundFile, format='wav')  ### save the new generated file in a folder
    else:
        print('Invalid distance parameters. d1 should be <= d2.')

In [18]:
def change_amplitude_range(emotionfile, newSoundFile, threshold):
    #amount = np.random.randint(0, threshold)
    amount = random.uniform(0, threshold)
    #print('Deamplify ' + str(emotionfile) + ' by ' + str(amount) + ' db.')
    sound = AudioSegment.from_file(emotionfile) - amount
    sound.export(newSoundFile, format='wav')  ### save the new generated file in a folder
    return amount

In [19]:
'''
def deamplify_per_folder(directory):
    for file in os.listdir(directory):
        if file.endswith('.wav'):
            soundFile = directory + file
            newSoundFile = directory + 'deamplified_' + file
            change_amplitude_range(soundFile, newSoundFile, 12)
'''

"\ndef deamplify_per_folder(directory):\n    for file in os.listdir(directory):\n        if file.endswith('.wav'):\n            soundFile = directory + file\n            newSoundFile = directory + 'deamplified_' + file\n            change_amplitude_range(soundFile, newSoundFile, 12)\n"

In [20]:
'''
for directory in emotion_train_folders:
    deamplify_per_folder(directory)
    
for directory in emotion_test_folders:
    deamplify_per_folder(directory)
'''

'\nfor directory in emotion_train_folders:\n    deamplify_per_folder(directory)\n    \nfor directory in emotion_test_folders:\n    deamplify_per_folder(directory)\n'

In [21]:
def add_noise_per_file(emotionfile, bgnoise, newSoundFile):
    
    emotionsound = AudioSegment.from_wav(emotionfile)
    emotion_duration = emotionsound.duration_seconds * 1000
    noise = AudioSegment.from_wav(bgnoise)
    noise_duration = noise.duration_seconds * 1000
    
    threshold = noise_duration - emotion_duration
    
    if threshold > 0:
        overlay_start = np.random.randint(0, threshold)
    else:
        overlay_start = 0
        
    targeted_chunk = noise[overlay_start:overlay_start + emotion_duration]
    newSound = emotionsound.overlay(targeted_chunk, position=0)
    newSound=newSound[0:5000]
    newSound.export(newSoundFile, format='wav')  ### save the new generated file in a folder

In [22]:
'''
def add_noise_per_folder(directory, extension, noise_directory):
    for file in os.listdir(directory):
        if file.endswith(extension) and not file[1] == '_':
            for i in range(0, 3):
                soundFile = directory + file
                newSoundFile = directory + str(i) + '_' + file
                random_noise = noise_directory + random.choice(os.listdir(noise_directory))

                add_noise_per_file(soundFile, random_noise, newSoundFile)
                print(newSoundFile)
'''

"\ndef add_noise_per_folder(directory, extension, noise_directory):\n    for file in os.listdir(directory):\n        if file.endswith(extension) and not file[1] == '_':\n            for i in range(0, 3):\n                soundFile = directory + file\n                newSoundFile = directory + str(i) + '_' + file\n                random_noise = noise_directory + random.choice(os.listdir(noise_directory))\n\n                add_noise_per_file(soundFile, random_noise, newSoundFile)\n                print(newSoundFile)\n"

In [24]:
def add_noise_and_deamplify_per_folder(directory, extension, noise_directory):
    for file in os.listdir(directory):
        if file.endswith(extension) and not file[1] == '_':
            for i in range(0, 2):
                soundFile = directory + file
                amount = change_amplitude_range(soundFile, soundFile, 12)
                noise = random.choice(os.listdir(noise_directory))
                random_noise = noise_directory + noise
                newSoundFile = directory + 'deamp_' + str(amount) + '_noise_' + noise[:len(noise)-5] + '_' + file
                add_noise_per_file(soundFile, random_noise, newSoundFile)
                print(newSoundFile)

            

In [ ]:
add_noise_and_deamplify_per_folder(h_directory, 'wav', background_noise_all)
add_noise_and_deamplify_per_folder(a_directory, 'wav', background_noise_all)
add_noise_and_deamplify_per_folder(n_directory, 'wav', background_noise_all)
add_noise_and_deamplify_per_folder(s_directory, 'wav', background_noise_all)
add_noise_and_deamplify_per_folder(o_directory, 'wav', background_noise_all)

D://Datasets//TRAINING//Happy//deamp_11.711286087788492_noise_beach_20_Happy374.wav
D://Datasets//TRAINING//Happy//deamp_2.62042774885282_noise_home_25_Happy374.wav
D://Datasets//TRAINING//Happy//deamp_0.39399263089444014_noise_residential_area__Happy375.wav
D://Datasets//TRAINING//Happy//deamp_0.4562591184360416_noise_home_83_Happy375.wav
D://Datasets//TRAINING//Happy//deamp_2.377523297887326_noise_residential_area__Happy376.wav
D://Datasets//TRAINING//Happy//deamp_3.793072747119846_noise_grocery_store_33_Happy376.wav
D://Datasets//TRAINING//Happy//deamp_11.875813638178908_noise_tram_66_Happy377.wav
D://Datasets//TRAINING//Happy//deamp_7.468908385851165_noise_city_center_106_Happy377.wav
D://Datasets//TRAINING//Happy//deamp_2.0123685762826393_noise_train_62_Happy378.wav
D://Datasets//TRAINING//Happy//deamp_7.266123296030791_noise_office_116_Happy378.wav
D://Datasets//TRAINING//Happy//deamp_5.520226580899562_noise_forest_path_47_Happy379.wav
D://Datasets//TRAINING//Happy//deamp_9.47986

D://Datasets//TRAINING//Happy//deamp_4.445665595662522_noise_metro_station_17_Happy421.wav
D://Datasets//TRAINING//Happy//deamp_0.23402722261216002_noise_park_75_Happy422.wav
D://Datasets//TRAINING//Happy//deamp_3.4412772105949223_noise_bus_38_Happy422.wav
D://Datasets//TRAINING//Happy//deamp_9.317877147063129_noise_cafe_restaurant_6_Happy423.wav
D://Datasets//TRAINING//Happy//deamp_9.947142835209842_noise_park_74_Happy423.wav
D://Datasets//TRAINING//Happy//deamp_7.1368497442810686_noise_park_17_Happy424.wav
D://Datasets//TRAINING//Happy//deamp_5.780600565369122_noise_library_84_Happy424.wav
D://Datasets//TRAINING//Happy//deamp_3.0440446366381075_noise_forest_path_38_Happy425.wav
D://Datasets//TRAINING//Happy//deamp_5.132223843470491_noise_train_63_Happy425.wav
D://Datasets//TRAINING//Happy//deamp_2.0366979561212033_noise_forest_path_48_Happy426.wav
D://Datasets//TRAINING//Happy//deamp_4.1587249407252695_noise_library_26_Happy426.wav
D://Datasets//TRAINING//Happy//deamp_4.8313996452028

D://Datasets//TRAINING//Happy//deamp_2.379939464408405_noise_residential_area_19_Happy469.wav
D://Datasets//TRAINING//Happy//deamp_10.829299202005327_noise_cafe_restaurant_33_Happy470.wav
D://Datasets//TRAINING//Happy//deamp_0.46844775872867217_noise_train_42_Happy470.wav
D://Datasets//TRAINING//Happy//deamp_2.2279904793068446_noise_cafe_restaurant_53_Happy471.wav
D://Datasets//TRAINING//Happy//deamp_11.602450178718247_noise_bus_62_Happy471.wav
D://Datasets//TRAINING//Happy//deamp_4.784756153464304_noise_library_84_Happy473.wav
D://Datasets//TRAINING//Happy//deamp_0.043296870683407374_noise_park_13_Happy473.wav
D://Datasets//TRAINING//Happy//deamp_10.59854208992179_noise_office_92_Happy474.wav
D://Datasets//TRAINING//Happy//deamp_4.56088758160072_noise_metro_station_17_Happy474.wav
D://Datasets//TRAINING//Happy//deamp_3.5647711327910545_noise_car_93_Happy475.wav
D://Datasets//TRAINING//Happy//deamp_10.25139698878775_noise_car_108_Happy475.wav
D://Datasets//TRAINING//Happy//deamp_10.498

D://Datasets//TRAINING//Happy//deamp_11.00808624932798_noise_park_14_Happy521.wav
D://Datasets//TRAINING//Happy//deamp_2.3607618917081576_noise_home_80_Happy522.wav
D://Datasets//TRAINING//Happy//deamp_10.277100170388012_noise_metro_station_68_Happy522.wav
D://Datasets//TRAINING//Happy//deamp_0.4607222976385601_noise_cafe_restaurant_39_Happy523.wav
D://Datasets//TRAINING//Happy//deamp_4.374472516522319_noise_grocery_store_51_Happy523.wav
D://Datasets//TRAINING//Happy//deamp_0.7358304992989826_noise_park_20_Happy524.wav
D://Datasets//TRAINING//Happy//deamp_5.385206451670037_noise_residential_area_71_Happy524.wav
D://Datasets//TRAINING//Happy//deamp_9.731678986251868_noise_tram_6_Happy525.wav
D://Datasets//TRAINING//Happy//deamp_8.109246562413102_noise_car_103_Happy525.wav
D://Datasets//TRAINING//Happy//deamp_7.79962493039629_noise_forest_path_28_Happy526.wav
D://Datasets//TRAINING//Happy//deamp_8.995827638909393_noise_city_center_110_Happy526.wav
D://Datasets//TRAINING//Happy//deamp_10.

D://Datasets//TRAINING//Happy//deamp_8.034603078156241_noise_tram_66_Happy570.wav
D://Datasets//TRAINING//Happy//deamp_0.3068931868171889_noise_library_24_Happy570.wav
D://Datasets//TRAINING//Happy//deamp_11.44855072269334_noise_library_85_Happy571.wav
D://Datasets//TRAINING//Happy//deamp_3.888994106456878_noise_bus_112_Happy571.wav
D://Datasets//TRAINING//Happy//deamp_10.456170409063024_noise_tram_65_Happy573.wav
D://Datasets//TRAINING//Happy//deamp_2.574449074236713_noise_office_57_Happy573.wav
D://Datasets//TRAINING//Happy//deamp_5.353430173949308_noise_library_86_Happy574.wav
D://Datasets//TRAINING//Happy//deamp_9.317745986965804_noise_metro_station_6_Happy574.wav
D://Datasets//TRAINING//Happy//deamp_10.834820975696795_noise_park_17_Happy575.wav
D://Datasets//TRAINING//Happy//deamp_3.3046319499627566_noise_park_76_Happy575.wav
D://Datasets//TRAINING//Happy//deamp_2.105454493113467_noise_park_75_Happy576.wav
D://Datasets//TRAINING//Happy//deamp_5.465404540042765_noise_cafe_restauran

...

D://Datasets//TRAINING//Happy//deamp_2.1354067755076116_noise_library_84_Happy1050.wav
D://Datasets//TRAINING//Happy//deamp_1.0531661127292877_noise_train_29_Happy1050.wav
D://Datasets//TRAINING//Happy//deamp_2.856028678648965_noise_bus_114_Happy1051.wav
D://Datasets//TRAINING//Happy//deamp_0.8244600361970815_noise_grocery_store_31_Happy1051.wav
D://Datasets//TRAINING//Happy//deamp_6.0678871488936945_noise_metro_station_68_Happy1052.wav
D://Datasets//TRAINING//Happy//deamp_5.392604739554535_noise_residential_area_5_Happy1052.wav
D://Datasets//TRAINING//Happy//deamp_1.6576262986040624_noise_car_25_Happy1053.wav
D://Datasets//TRAINING//Happy//deamp_7.6942813623240465_noise_forest_path_38_Happy1053.wav
D://Datasets//TRAINING//Happy//deamp_1.6384049218440224_noise_park_15_Happy1054.wav
D://Datasets//TRAINING//Happy//deamp_0.1528422197798447_noise_bus_114_Happy1054.wav
D://Datasets//TRAINING//Happy//deamp_5.78576218710751_noise_cafe_restaurant_11_Happy1055.wav
D://Datasets//TRAINING//Happy/

D://Datasets//TRAINING//Happy//deamp_11.716367869944678_noise_train_64_Happy1095.wav
D://Datasets//TRAINING//Happy//deamp_8.052825489133236_noise_cafe_restaurant_11_Happy1095.wav
D://Datasets//TRAINING//Happy//deamp_6.835445199332364_noise_metro_station_46_Happy1096.wav
D://Datasets//TRAINING//Happy//deamp_5.058933620751458_noise_car_15_Happy1096.wav
D://Datasets//TRAINING//Happy//deamp_2.7876688806502936_noise_home_22_Happy1097.wav
D://Datasets//TRAINING//Happy//deamp_6.103853675429596_noise_residential_area_9_Happy1097.wav
D://Datasets//TRAINING//Happy//deamp_6.050256154961306_noise_train_63_Happy1098.wav
D://Datasets//TRAINING//Happy//deamp_3.6354859422736916_noise_metro_station_31_Happy1098.wav
D://Datasets//TRAINING//Happy//deamp_3.62822414114167_noise_library_26_Happy1099.wav
D://Datasets//TRAINING//Happy//deamp_0.24226048371607023_noise_park_19_Happy1099.wav
D://Datasets//TRAINING//Happy//deamp_0.6365374881571411_noise_cafe_restaurant_90_Happy1100.wav
D://Datasets//TRAINING//Hap

D://Datasets//TRAINING//Happy//deamp_11.513654226106663_noise_park_76_Happy1141.wav
D://Datasets//TRAINING//Happy//deamp_2.4484328168641962_noise_residential_area_69_Happy1141.wav
D://Datasets//TRAINING//Happy//deamp_5.81621042745344_noise_cafe_restaurant_44_Happy1142.wav
D://Datasets//TRAINING//Happy//deamp_8.548966586519054_noise_bus_112_Happy1142.wav
D://Datasets//TRAINING//Happy//deamp_8.82632544696476_noise_car_89_Happy1143.wav
D://Datasets//TRAINING//Happy//deamp_5.2485867167746605_noise_forest_path_40_Happy1143.wav
D://Datasets//TRAINING//Happy//deamp_1.4643641995039975_noise_beach_20_Happy1144.wav
D://Datasets//TRAINING//Happy//deamp_1.6475003569909745_noise_cafe_restaurant_58_Happy1144.wav
D://Datasets//TRAINING//Happy//deamp_1.6322048543150567_noise_park_16_Happy1145.wav
D://Datasets//TRAINING//Happy//deamp_7.353677323318953_noise_bus_3_Happy1145.wav
D://Datasets//TRAINING//Happy//deamp_5.00733565929091_noise_home_88_Happy1146.wav
D://Datasets//TRAINING//Happy//deamp_2.505373

D://Datasets//TRAINING//Happy//deamp_7.337050579206801_noise_tram_65_Happy1187.wav
D://Datasets//TRAINING//Happy//deamp_10.708123534598464_noise_metro_station_46_Happy1187.wav
D://Datasets//TRAINING//Happy//deamp_6.459380347673713_noise_park_76_Happy1188.wav
D://Datasets//TRAINING//Happy//deamp_1.2214567541637606_noise_cafe_restaurant_27_Happy1188.wav
D://Datasets//TRAINING//Happy//deamp_1.8403368398836069_noise_park_18_Happy1189.wav
D://Datasets//TRAINING//Happy//deamp_6.097607987785384_noise_grocery_store_31_Happy1189.wav
D://Datasets//TRAINING//Happy//deamp_1.481400076998363_noise_residential_area_70_Happy1190.wav
D://Datasets//TRAINING//Happy//deamp_3.6524769474381533_noise_grocery_store_101_Happy1190.wav
D://Datasets//TRAINING//Happy//deamp_3.0118009492519184_noise_car_104_Happy1191.wav
D://Datasets//TRAINING//Happy//deamp_4.951609446938761_noise_forest_path_28_Happy1191.wav
D://Datasets//TRAINING//Happy//deamp_4.607348644172335_noise_grocery_store_103_Happy1192.wav
D://Datasets//

D://Datasets//TRAINING//Happy//deamp_10.612126756426594_noise_metro_station_17_Happy658.wav
D://Datasets//TRAINING//Happy//deamp_9.20749778840818_noise_bus_111_Happy658.wav
D://Datasets//TRAINING//Happy//deamp_8.316462121553407_noise_cafe_restaurant_93_Happy1231.wav
D://Datasets//TRAINING//Happy//deamp_6.835388629807807_noise_office_116_Happy1231.wav
D://Datasets//TRAINING//Happy//deamp_1.991633157672744_noise_metro_station_17_Happy1232.wav
D://Datasets//TRAINING//Happy//deamp_1.4131267919982462_noise_train_30_Happy1232.wav
D://Datasets//TRAINING//Happy//deamp_5.912803333925082_noise_metro_station_98_Happy1233.wav
D://Datasets//TRAINING//Happy//deamp_11.806126554857439_noise_library_89_Happy1233.wav
D://Datasets//TRAINING//Happy//deamp_9.68135845137355_noise_residential_area_72_Happy1234.wav
D://Datasets//TRAINING//Happy//deamp_3.0604661032127902_noise_home_26_Happy1234.wav
D://Datasets//TRAINING//Happy//deamp_7.8487900123647805_noise_residential_area_8_Happy1235.wav
D://Datasets//TRAI

In [ ]:
percent = 0.2
partition_directory(h_directory, h_test, percent)
partition_directory(a_directory, a_test, percent)
partition_directory(n_directory, n_test, percent)
partition_directory(s_directory, s_test, percent)
partition_directory(o_directory, o_test, percent)

In [ ]:
add_noise_and_deamplify_per_folder(h_test, '.wav', background_noise_test)

In [ ]:
add_noise_and_deamplify_per_folder(a_test, '.wav', background_noise_test)

In [ ]:
add_noise_and_deamplify_per_folder(n_test, '.wav', background_noise_test)

In [ ]:
add_noise_and_deamplify_per_folder(s_test, '.wav', background_noise_test)

In [ ]:
add_noise_and_deamplify_per_folder(o_test, '.wav', background_noise_test)

In [ ]:
# AudioEffectsChain().reverb()(infile, outfile)